## WMLMIA 2018
## Tutorial 1b : Python for MedIA Lab

In [ ]:
%matplotlib inline
import scipy.io as sio
import numpy as np
from PIL import Image
from myFcnCart2Pol import myFcnCart2Pol
from skimage.segmentation import random_walker
import skimage
import scipy.signal as signal
import scipy.ndimage.morphology as morph
from myFcnPol2Cart import myFcnPol2Cart
import pickle
import matplotlib.pyplot as plt

In [ ]:
# Size of image in polar coordinates
szPol = 256

# Window size for feature extraction
Kernelmat = np.array([3,5,7,13,17,21,25,30])

In [ ]:
# Loading trained Ranfom-Forest model for segmentation
pkl_file = open('data/day1/tut01/myfile.pkl','rb')
rf = pickle.load(pkl_file,encoding='latin1')
pkl_file.close()

In [ ]:
# Load original image and ground truth
OrgImage = np.asarray(Image.open('data/day1/tut01/orig_frame_01_0018_003.png').convert('L'))
GtImage = np.asarray(Image.open('data/day1/tut01/gt_frame_01_0018_003.png').convert('L'))
# Size of image
(szCart1, szCart2) = OrgImage.shape
# Convert images from cartesian to polar
polarOrgImage = myFcnCart2Pol(OrgImage, szPol, szPol)
polarGtImage = myFcnCart2Pol(GtImage, szPol, szPol)

In [ ]:
plt.subplot(2,2,1)
plt.imshow(OrgImage,'gray')
plt.subplot(2,2,2)
plt.imshow(GtImage,'gray')
plt.subplot(2,2,3)
plt.imshow(polarOrgImage,'gray')
plt.subplot(2,2,4)
plt.imshow(polarGtImage,'gray')

In [ ]:
# Feature Extraction
beta = 2000 
markers = np.zeros(polarOrgImage.shape, dtype=np.uint) # seed points   
markers[0,:] = 2
markers[szPol-1,:] = 1
labels = random_walker(skimage.img_as_float(polarOrgImage), markers, beta, mode='bf', copy=True, return_full_prob=True)

for scaleCtr in range(0,len(Kernelmat)):
    hScale = np.ones((Kernelmat[scaleCtr], Kernelmat[0]))
    hScale = hScale/np.sum(hScale)
    ConvMean  = signal.convolve2d(polarOrgImage, hScale, mode='same')
    SqMean = signal.convolve2d(polarOrgImage**2, hScale, mode='same')
    ConvVar = SqMean - ConvMean**2
    if scaleCtr == 0:
        MeanSet = np.reshape(ConvMean, (1,np.product(ConvMean.shape)))[0]
        VarSet = np.reshape(ConvVar, (1,np.product(ConvVar.shape)))[0]
        ConfSet = np.reshape(labels[1,:,:], (1,np.product(labels[1,:,:].shape)))[0]
    else:
        MeanSet = np.vstack((MeanSet,np.reshape(ConvMean, (1,np.product(ConvMean.shape)))[0]))
        VarSet = np.vstack((VarSet,np.reshape(ConvVar, (1,np.product(ConvVar.shape)))[0]))

TestingFeatureSet = np.zeros((np.product(polarOrgImage.shape),17)) # No. of features=17
MeanSet = MeanSet.transpose()
VarSet = VarSet.transpose()
TestingFeatureSet[:,0:16] = np.hstack((MeanSet,VarSet))
TestingFeatureSet[:,16] = ConfSet

In [ ]:
# Prediction from trained random forest model
Pred = rf.predict(TestingFeatureSet[:,0:17])
PredImage = np.reshape(Pred,(szPol,szPol))

In [ ]:
# Separating contours for lumen and media
PredBwLum = np.zeros(PredImage.shape)     
PredBwLum[PredImage == 1] = 1
PredBwMed = np.zeros(PredImage.shape)     
PredBwMed[(PredImage ==1) | (PredImage ==2)] = 1

In [ ]:
# Morphological operations
LumBw = skimage.morphology.dilation((skimage.morphology.erosion(PredBwLum, skimage.morphology.disk(2))), skimage.morphology.disk(2))
LumBw = skimage.morphology.label(LumBw)
MedBw = skimage.morphology.dilation((skimage.morphology.erosion(PredBwMed, skimage.morphology.disk(2))), skimage.morphology.disk(2))
MedBw = skimage.morphology.label(MedBw)
LumBw[np.nonzero(LumBw >1)] = 0
MedBw[np.nonzero(MedBw >1)] = 0
LumBw = morph.binary_fill_holes(LumBw)
MedBw = morph.binary_fill_holes(MedBw)

In [ ]:
# Converting back to cartesian
Lum = myFcnPol2Cart(LumBw, szCart1, szCart2)
Med = myFcnPol2Cart(MedBw, szCart1, szCart2)

In [ ]:
k = 15 # size of structural elemant
Lum = skimage.morphology.erosion((skimage.morphology.dilation(Lum, skimage.morphology.disk(k))), skimage.morphology.disk(k))
Lum = skimage.morphology.label(Lum)
Lum[np.nonzero(Lum >1)] = 0
Lum = morph.binary_fill_holes(Lum)
Med = skimage.morphology.erosion((skimage.morphology.dilation(Med, skimage.morphology.disk(k))), skimage.morphology.disk(k))
Med = morph.binary_fill_holes(Med)
FedgeLum = skimage.morphology.dilation(Lum, skimage.morphology.disk(3)) ^ Lum
FedgeMed = skimage.morphology.dilation(Med, skimage.morphology.disk(3)) ^ Med

In [ ]:
GtLum = np.zeros((GtImage.shape))
GtMed = np.zeros((GtImage.shape))
GtLum[GtImage == 254] = 1
GtMed[GtImage > 0] = 1
GtEdgeLum = skimage.morphology.dilation(GtLum, skimage.morphology.disk(3)) - GtLum
GtEdgeMed = skimage.morphology.dilation(GtMed, skimage.morphology.disk(3)) - GtMed

In [ ]:
# Color mapping
Org = np.empty((szCart1, szCart2, 3), dtype=np.uint8)
Org[:, :, 0] = Org[:, :, 1] = Org[:, :, 2] = OrgImage
Org[GtEdgeLum == 1,0] = 0; Org[GtEdgeLum == 1,1] = 0; Org[GtEdgeLum == 1,2] = 255
Org[GtEdgeMed == 1,0] = 255; Org[GtEdgeMed == 1,1] = 255; Org[GtEdgeMed == 1,2] = 0
Org[FedgeLum == 1,0] = 255; Org[FedgeLum == 1,1] = 0; Org[FedgeLum == 1,2] = 0
Org[FedgeMed == 1,0] = 0; Org[FedgeMed == 1,1] = 255; Org[FedgeMed == 1,2] = 0

In [ ]:
OrgI = Image.fromarray((Org).astype(np.uint8));
OrgI.show()